In [1]:
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

#### Hyperparameter

- epoch : 데이터셋을 반복하는 횟수
- batch size : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
- learning rate : 각 batch/epoch에서 모델의 매개변수를 조절하는 비율

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Optimization Loop(최적화 단계)

#### 한 epoch의 구성

- 학습 단계(train loop) : 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴
- 검증/테스트 단계(validation/test loop) : 모델 성능이 개선되고 있는지 확인하기 위해 테스트 데이터셋 반복

#### 학습 단계에서의 최적화
- optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정

- loss.backward()를 호출하여 예측 손실(prediction loss)을 역전파 -> PyTorch는 각 매개변수에 대한 손실의 변화도를 저장

- 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정

In [3]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [4]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297871  [    0/60000]
loss: 2.292700  [ 6400/60000]
loss: 2.289830  [12800/60000]
loss: 2.299739  [19200/60000]
loss: 2.280023  [25600/60000]
loss: 2.274681  [32000/60000]
loss: 2.280801  [38400/60000]
loss: 2.261215  [44800/60000]
loss: 2.260773  [51200/60000]
loss: 2.278499  [57600/60000]
Test Error: 
 Accuracy: 29.4%, Avg loss: 2.260188 

Epoch 2
-------------------------------
loss: 2.239497  [    0/60000]
loss: 2.234810  [ 6400/60000]
loss: 2.225440  [12800/60000]
loss: 2.270161  [19200/60000]
loss: 2.213130  [25600/60000]
loss: 2.208396  [32000/60000]
loss: 2.219394  [38400/60000]
loss: 2.184003  [44800/60000]
loss: 2.190847  [51200/60000]
loss: 2.210628  [57600/60000]
Test Error: 
 Accuracy: 29.9%, Avg loss: 2.182242 

Epoch 3
-------------------------------
loss: 2.149533  [    0/60000]
loss: 2.127820  [ 6400/60000]
loss: 2.109111  [12800/60000]
loss: 2.181223  [19200/60000]
loss: 2.083321  [25600/60000]
loss: 2.072659  [32000/600